In [ ]:
import fun.em.em as em
from util.math.linear import (
    column_vector, column_indicator, inner_product
)

import numpy as np
from numpy.linalg import inv
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import patsy

# Import Sample Data from ISLP

In [ ]:
from ISLP import load_data

In [ ]:
# We use the Boston Housing Data from the module ISLP.
# medv is median house value, what we want to predict.
# 
# We only select subset of predictors.
# rm: average number of rooms
# age: proportion of owner-occupied units built prior to 1940
# lstat: percentage of households with low socioeconomic status

boston = load_data("Boston")
boston.head()

In [ ]:
# We use patsy for dmatrices, which behaves similarly as numpy
X, A = patsy.dmatrices("medv~rm+age+lstat-1", boston) #we use X, A to be consistent with the notes.
print(X.shape)
print(A.shape)
beta_lr = inv(A.T@A)@A.T@X
print("Trained Beta", np.round(beta_lr.T, 2))

# Censored Regression
## Fake Data Generation
Following the book, $X = A\beta + U$. But only $Y_i = \min(X_i, c_i)$ is observed.

In [ ]:
np.random.seed(2)
n = X.shape[0]
beta = column_vector([3, 2, -0.9])
sigma = 10
U = column_vector(np.random.normal(0, 1, n))

X_true = A@beta + sigma * U

In [ ]:
np.random.seed(2)
c = column_vector(np.random.binomial(1, 0.5, n)) * 0.7 # 40% cencsored to be 70%
c[c == 0] = X_true[c == 0] * 10 # censored
C = X_true * c 
Y = np.minimum(X_true, C  ) # IMPORTANT!! Assumed that all X_true are positive

In [ ]:
plt.plot(X_true)
plt.plot(np.where(Y< X_true, Y, -np.inf))

## Train with Linear Regression Method

In [ ]:
beta_lr = inv(A.T@A)@A.T@Y
print("Trained", np.round(beta_lr.T, 2))
print("True", beta.T)
print("It differs quite a lot.")

## Train with Censored Regression Method

In [ ]:
def get_H(v):
    a =  norm.pdf(v) / (1 - norm.cdf(v))
    return a

def get_Hn(c, A, beta_n, sigma_n, In):
    return get_H((c - A@beta_n)/sigma_n * In)

def get_In(C, Y):
    return column_indicator(C == Y)

def get_bar_S(C, A, beta_n, sigma_n, In, Hn):
    out = (A@beta_n + (sigma_n**2) * np.ones(C.shape) + sigma_n* (C + A@beta_n) * Hn) * In
    return np.where(np.isnan(out), 0, out)

def get_S(Y, In):
    return Y * (1 - In)

def get_expectation_X(A, beta_n, sigma_n, In, Hn, S):
    temp = (A@beta_n + sigma_n * Hn) * In
    temp = np.where(np.isnan(temp), 0, temp)
    return temp + S

def get_expectation_X2(bar_S, S):
    return (inner_product(bar_S, bar_S) + inner_product(S, S))

In [ ]:
bar_S = get_bar_S(C, A, beta_lr, sigma, In, Hn)
S = get_S(Y, In)

In [ ]:
E_X = get_expectation_X(A, beta_lr, sigma, In, Hn, S)

In [ ]:
beta_np1 = inv(A.T@A)@A.T@E_X

In [ ]:
def update_em(data, theta):
    # Reading data
    y, c, A = data
    m = A.shape[1]

    # Load paramter
    beta_n, sigma2_n = column_vector(theta[:m]), theta[m]
    sigma_n = np.sqrt(sigma2_n)

    # Compute
    In = get_In(c, y)
    
    Hn =  get_Hn(c, A, beta_n, sigma_n, In)
    Hn = np.where(np.isinf(Hn), 3, Hn)

    # Compute S
    bar_S = get_bar_S(C, A, beta_n, sigma, In, Hn)
    S = get_S(Y, In)

    # Compute Expectation
    E_X = get_expectation_X(A, beta_n, sigma, In, Hn, S)
    E_X2 = get_expectation_X2(bar_S, S)
    

    beta_np1 = inv(A.T@A)@A.T@E_X

    # Compute
    r = Y.shape[0]
    sigma2_np1 =  1/r * (
            inner_product(A@beta_np1, A@beta_np1) 
            - 2 * inner_product( A@beta_np1,E_X)
            + E_X2
    )
    theta_np1 = np.append(beta_np1, sigma2_np1)
    print(theta_np1)
    return theta_np1

In [ ]:
data = (Y, C, A)
theta0 = np.append(beta_lr, 30)
theta = theta0
print("theta0", theta0)
print("theta0", beta.T)

In [ ]:
y, c, A = data
m = A.shape[1]

# Load paramter
beta_n, sigma2_n = column_vector(theta[:m]), theta[m]
sigma_n = np.sqrt(sigma2_n)

# Compute
In = get_In(c, y)
Hn =  get_Hn(c, A, beta_n, sigma_n, In)
Hn = np.where(np.isinf(Hn), 3, Hn)

# Compute S
bar_S = get_bar_S(C, A, beta_n, sigma, In, Hn)
S = get_S(Y, In)

# Compute Expectation
E_X = get_expectation_X(A, beta_n, sigma, In, Hn, S)
E_X2 = get_expectation_X2(bar_S, S)

In [ ]:
model = em.EM(n_iter = 200)
model.load(data, update_em)
a = model.fit(theta0)

In [ ]:
plt.plot(a)
plt.grid(True)